In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.5'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=',', header = True, ignoreLeadingWhiteSpace = True)
home_df.head(5)


[Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date='2022-04-08', date_built='2016', price='936923', bedrooms='4', bathrooms='3', sqft_living='3167', sqft_lot='11733', floors='2', waterfront='1', view='76'),
 Row(id='7530a2d8-1ae3-4517-9f4a-befe060c4353', date='2021-06-13', date_built='2013', price='379628', bedrooms='2', bathrooms='2', sqft_living='2235', sqft_lot='14384', floors='1', waterfront='0', view='23'),
 Row(id='43de979c-0bf0-4c9f-85ef-96dc27b258d5', date='2019-04-12', date_built='2014', price='417866', bedrooms='2', bathrooms='2', sqft_living='2127', sqft_lot='10575', floors='2', waterfront='0', view='0'),
 Row(id='b672c137-b88c-48bf-9f18-d0a4ac62fb8b', date='2019-10-16', date_built='2016', price='239895', bedrooms='2', bathrooms='2', sqft_living='1631', sqft_lot='11149', floors='2', waterfront='0', view='0'),
 Row(id='e0726d4d-d595-4074-8283-4139a54d0d63', date='2022-01-08', date_built='2017', price='424418', bedrooms='3', bathrooms='2', sqft_living='2249', sqft_lot='13878

In [ ]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView('home_sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_4bd = spark.sql("SELECT YEAR(date), ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 4 GROUP BY YEAR(date) ORDER BY YEAR(date)")
avg_price_4bd.show()

+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



2019: $300263.70

---
2020: $298353.78

---
2021: $301819.44

---
2022: $296363.88

In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3bd_3ba = spark.sql("SELECT YEAR(date_built), ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY YEAR(date_built) ORDER BY YEAR(date_built)")
avg_price_3bd_3ba.show()

+----------------+---------+
|year(date_built)|avg_price|
+----------------+---------+
|            2010|292859.62|
|            2011|291117.47|
|            2012|293683.19|
|            2013|295962.27|
|            2014|290852.27|
|            2015| 288770.3|
|            2016|290555.07|
|            2017|292676.79|
+----------------+---------+



2010: $292859.62

---
2011: $291117.47

---
2012: $293683.19

---
2013: $295962.27

---
2014: $290852.27

---
2015: $288770.30

---
2016: $290555.07

---
2017: $292676.79

In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_3bd_3ba_2fl = spark.sql("SELECT YEAR(date_built), ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY YEAR(date_built) ORDER BY YEAR(date_built)")
avg_price_3bd_3ba_2fl.show()

+----------------+---------+
|year(date_built)|avg_price|
+----------------+---------+
|            2010|285010.22|
|            2011|276553.81|
|            2012|307539.97|
|            2013|303676.79|
|            2014|298264.72|
|            2015|297609.97|
|            2016| 293965.1|
|            2017|280317.58|
+----------------+---------+



2010: $285010.22

---
2011: $276553.81

---
2012: $307539.97

---
2013: $303676.79

---
2014: $298264.72

---
2015: $297609.97

---
2016: $293965.10

---
2017: $280317.58

In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales GROUP BY view HAVING AVG(price) >= 350000 ORDER BY view DESC").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.44020986557006836 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('home_sales')

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales GROUP BY view HAVING AVG(price) >= 350000 ORDER BY view DESC").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.8729970455169678 seconds ---


In [ ]:
# runtime for cached: 0.8729970455169678 seconds
# runtime for uncached: 0.44020986557006836 seconds
# runtime for cached is longer than uncached by .43 seconds

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.parquet("home_sales_partition", partitionBy = "date_built")

In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_partition")
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_home_sales')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM parquet_home_sales GROUP BY view HAVING AVG(price) >= 350000 ORDER BY view DESC").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.7555925846099854 seconds ---


In [ ]:
# parquet runtime: 0.7555925846099854 seconds
# cached runtime: 0.8729970455169678 seconds
# parquet runtime is faster than cached by .12 seconds

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False